# Quadrotor (aka "the drone")

Import modules and configure the notebook.

In [2]:
# import required libraries
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import ae353_drone
import importlib
importlib.reload(ae353_drone)

<module 'ae353_drone' from '/home/pranay/Downloads/AE353/DP4/ae353_drone.py'>

## Synthesize Linearized System Model

### Derive EOM

In [ ]:
# copy from DeriveEOM notebook

In [ ]:
# get f

In [ ]:
# think about what g should be.
    # y = g(x), i.e., g should be the relation between state and output. What if output is directly some of the states?
    # what is the dimension of g? i.e., how many equations in g if say y is 6x1?

# get g

### Linearize - Get A, B, C, D matrices

In [ ]:
# pick state variables

# pick control variables 

# pick equilibrium values

In [ ]:
# verify that equilibrium values indeed satisfy f = 0
    # create a substitutable version of f and when you substitue equilibrium values in it, every element should be zero

In [ ]:
# Get A matrix

# do jacobian of f and substitute the eq values
print(f'Dimensions of A matrix is: {np.shape(A)}')

# Get B

# do jacobian of f and substitute the eq values
print(f'Dimensions of B matrix is: {np.shape(B)}')

In [ ]:
# Get C matrix

# do jacobian of g and substitute the eq values
print(f'Dimensions of C matrix is: {np.shape(C)}')

### Verify Controllability and Observability

In [ ]:
# controllability

# get controllability matrix W

print(f'Rank of W matrix is: {np.linalg.matrix_rank(W)}')
print(f'Number of state variables is: {A.shape[0]}')

In [ ]:
# observability

# get observability matrix O

print(f'Rank of O matrix is: {np.linalg.matrix_rank(O)}')
print(f'Number of state variables is: {A.shape[0]}')

## Design Controller and Observer

In [ ]:
# Design controller

print(f'Shape of K is: {np.shape(K)}')

In [ ]:
# design observer

print(f'Shape of L is: {np.shape(L)}')

## Run Simulation

In [ ]:
# create an instance of the simulator. Run this only once. You can run rest of the cells if you update the controller
# but don't run this cell again

# pos_noise = 0 makes noise in position measurements zero. It should be left at 0
# rpy_noise = 0 makes noise in orientation measurements zero. It should be left at 0
# num_rings = number of rings in the arena. Default value is 2.
# ring_separation = separation between rings. Leave it at 5.

simulator = ae353_drone.Simulator(display=True, pos_noise=0, rpy_noise=0, num_rings=2, ring_separation=5)

In [ ]:
class RobotController:
    def __init__(self, limiter=None):
        self.dt = 0.01
        self.limiter = limiter # function to provide lower and upper bounds on your control outputs value. 
                               # You don't need a limiter function so you can leave it unchanged. 

    def get_color(self):
        return [0., 1., 0.]

    def reset(self, pos):
        self.xhat = np.zeros(12).reshape((12,1))

    def run(self, pos, rpy, pos_ring, is_last_ring, pos_others):
        
        # Inputs description:
        # pos = current position of drone
        # rpy = current rpy of drone
        # pos_ring = position of next ring center
        # is_last_ring = true or false
        # pos_others = position of other drones, if exist
        
        # create y vector
        
        # get desired point and make it an equilibrium point
      
        # get control 
  
        # update state estimate

        # data is automatically logged; see plotting code below to see how to access it
        
        
        # setup a limiter function - don't change
        if self.limiter is not None:
            tau_x, tau_y, tau_z, f_z = self.limiter(tau_x, tau_y, tau_z, f_z)

        # return controls
        return tau_x, tau_y, tau_z, f_z

In [ ]:
# clear any previously added drones from the simulator. Run this cell to reset the environment
simulator.clear_drones()

Add the drone to the simulator. There are three arguments:

* `RobotController` is the class definition
* `'my_netid'` is the name of the drone
* `'my_image.png'` is the name of an image to use as a label (can be `None`, can be a file path)

You can call this function more than once, so long as you call it with a different drone name each time. An exception will be raised if you try to add two drones of the same name to the simulator.

In [ ]:
# add a drone that operates with the controller 'RobotController' that you designed above
simulator.add_drone(RobotController, 'my_netid', 'my_image.png')

Reset the drone (i.e., place it at a random location in the start ring).

In [ ]:
# reset drone to random initial location
simulator.reset() 

Run simulation until `max_time` is reached or until the drone reaches the finish ring.

In [ ]:
# simulation runs until max_time or until reaching last ring
simulator.run(max_time=5.0) 

## Example of how to get data and plot results

Assume that we already did two things:

* Added a drone with the name `my_netid` to the simulator
* Ran the simulation

Get drone by name. It is the name you assigned while using `simulator.add_drone()`. 

In [ ]:
drone_name = 'my_netid' # drone name
drone = simulator.get_drone_by_name(drone_name) # get the internal drone object corresponding to the drone name

if drone is None:
    drone_names = '\n'.join([d['name'] for d in simulator.drones])
    msg = f'The simulator has no drone with name "{drone_name}".'
    if len(drone_names) == 0:
        msg += f' The simulator has no drones at all, in fact.'
    else:
        msg += f' The simulator has these drones:'
        msg += f'\n==========\n{drone_names}\n==========\n'
    print(msg)

Extract data.

In [ ]:
# get all the data from the drone with the name 'my_netid' as assigned in previous cell
data = drone['data'].copy()

Convert all lists in data to numpy arrays.

In [ ]:
for key in data.keys():
    if key != 'user_data':
        data[key] = np.array(data[key]).T
for key in data['user_data'].keys():
    data['user_data'][key] = np.array(data['user_data'][key]).T

Did the drone finish? If so, what was the time at which it finished? If not, was it still running (or was there likely an error)?

In [ ]:
if drone['finish_time'] is None:
    if drone['running']:
        print(f'The drone "{drone["name"]}" did not finish.')
    else:
        print(f'The drone "{drone["name"]}" did not finish and was no longer running.')
else:
    print(f'The drone "{drone["name"]}" finished with time: {drone["finish_time"]}')

Plot results.

In [ ]:
# Create a figure with subplots that all share the same x-axis
fig, (ax_pos, ax_rpy, ax_act) = plt.subplots(3, 1, figsize=(9, 12), sharex=True)

# Position
ax_pos.plot(data['t'], data['pos'][0, :], label='x (m)', linewidth=4)
ax_pos.plot(data['t'], data['pos'][1, :], label='y (m)', linewidth=4)
ax_pos.plot(data['t'], data['pos'][2, :], label='z (m)', linewidth=4)
ax_pos.grid()
ax_pos.legend(fontsize=16)
ax_pos.tick_params(labelsize=14)

# Roll, pitch, and yaw angles
ax_rpy.plot(data['t'], data['rpy'][0, :], label='roll (rad)', linewidth=4)
ax_rpy.plot(data['t'], data['rpy'][1, :], label='pitch (rad)', linewidth=4)
ax_rpy.plot(data['t'], data['rpy'][2, :], label='yaw (rad)', linewidth=4)
ax_rpy.grid()
ax_rpy.legend(fontsize=16)
ax_rpy.tick_params(labelsize=14)

# Actuator commands
ax_act.plot(data['t'], data['tau_x'], label='tau_x (N-m)', linewidth=4)
ax_act.plot(data['t'], data['user_data']['tau_x_des'], '--', label='desired tau_x (N-m)', linewidth=4)
ax_act.plot(data['t'], data['tau_y'], label='tau_y (N-m)', linewidth=4)
ax_act.plot(data['t'], data['user_data']['tau_y_des'], '--', label='desired tau_y (N-m)', linewidth=4)
ax_act.plot(data['t'], data['tau_z'], label='tau_z (N-m)', linewidth=4)
ax_act.plot(data['t'], data['user_data']['tau_z_des'], '--', label='desired tau_z (N-m)', linewidth=4)
ax_act.plot(data['t'], data['f_z'], label='f_z (N)', linewidth=4)
ax_act.plot(data['t'], data['user_data']['f_z_des'], '--', label='desired f_z (N-m)', linewidth=4)
ax_act.grid()
ax_act.legend(fontsize=16)
ax_act.tick_params(labelsize=14)

# Set shared x-axis properties
ax_act.set_xlabel('time (s)', fontsize=20)
ax_act.set_xlim([data['t'][0], data['t'][-1]])

# Make the arrangement of subplots look nice
fig.tight_layout()

# Additional Code Snippets

### The following code may not be useful if you are not participating in the contests and you can delete it from your notebook

The following sections provide code snippets for doing different things, including making videos, rearranging the rings, loading controllers from the `students` folder to test your collision avoidance code, etc. 

## Example of how to load drones from the "students" directory

Load drones from `students` directory.

In [ ]:
simulator.load_drones()

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

Run simulation for `max_time` seconds or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

## Example of how to show results, change views, keep running

Show results (so far).

In [ ]:
simulator.show_results()

View from behind a particular drone.

In [ ]:
simulator.camera_droneview('template')

Continue running simulation from same point for `max_time` seconds or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

View from finish ring instead of from start ring.

In [ ]:
simulator.camera_finishview()

Continue running simulation from same point for `max_time` seconds or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

Show results (so far).

In [ ]:
simulator.show_results()

## Example of how to clear and reload drones

Clear all drones.

In [ ]:
simulator.clear_drones()

Load drones again from `students` directory.

In [ ]:
simulator.load_drones()

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

Run simulation until `max_time` is reached or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

## Example of how to rearrange the rings

Rearrange the rings.

In [ ]:
simulator.move_rings()

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

Run simulation until `max_time` is reached or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

## Example of how to take a snapshot

View from behind a particular drone.

In [ ]:
simulator.camera_droneview('template')

Display a snapshot.

In [ ]:
# Get snapshot as 480 x 480 x 4 numpy array of RGBA values
rgba = simulator.snapshot()

# Display snapshot
plt.figure(figsize=(9, 9))
plt.imshow(rgba)

Save the snapshot.

In [ ]:
plt.imsave('my_snapshot.png', rgba)

## Example of how to record a movie

You must install both [imageio](https://github.com/imageio/imageio) and [imageio-ffmpeg](https://github.com/imageio/imageio-ffmpeg) in order for this to work. You can do this from a terminal (in your `ae353-bullet` environment, for example) as follows:

```
pip install imageio
pip install imageio-ffmpeg
```

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

View from behind a particular drone.

In [ ]:
simulator.camera_droneview('template')

Run simulation until `max_time` is reached or until all drones finish, saving movie to `my_video.mp4`.

In [ ]:
simulator.run(max_time=1.0, video_filename='my_video.mp4')